# Using compression

## Load movielens datasets

In [1]:
import os
import numpy as np
import pandas as pd
import tables

In [2]:
# Import CSV files via pandas
dset = 'movielens-1m'
fdata = os.path.join(dset, 'ratings.dat.gz')
fitem = os.path.join(dset, 'movies.dat')

# pass in column names for each CSV
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(fdata, sep=';', names=r_cols, compression='gzip')

m_cols = ['movie_id', 'title', 'genres']
movies = pd.read_csv(fitem, sep=';', names=m_cols,
                     dtype={'title': object, 'genres': object})

In [3]:
movies.ftypes

movie_id     int64:dense
title       object:dense
genres      object:dense
dtype: object

In [4]:
ratings.ftypes

user_id           int64:dense
movie_id          int64:dense
rating            int64:dense
unix_timestamp    int64:dense
dtype: object

## Storing in HDF5/PyTables in compressed form

In [5]:
import os
import shutil
data_dir = "compression"
if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.mkdir(data_dir)

In [6]:
def to_hdf5(ratings, movies, filters):
    
    class Ratings(tables.IsDescription):
        user_id = tables.Int32Col(pos=0)
        movie_id = tables.Int32Col(pos=1)
        rating = tables.Int8Col(pos=2)
        unix_timestamp = tables.Int64Col(pos=3)
    
    class Movies(tables.IsDescription):
        movie_id = tables.Int32Col(pos=0)
        title = tables.StringCol(100, pos=1)
        genres = tables.StringCol(50, pos=2)
    
    def get_filename(filters):
        if filters.complevel != 0:
            complib = filters.complib if ":" not in filters.complib else filters.complib.replace(":", "-")
            shuffle = "shuffle" if filters.shuffle else "noshuffle"
            filename = "%s/%s-%d-%s.h5" % (data_dir, complib, filters.complevel, shuffle)
        else:
            filename = "%s/no-compressed.h5" % (data_dir,)
        return filename

    filename = get_filename(filters)
    print("Creating file:", filename)
    with tables.open_file(filename, "w") as f:
        table_ratings = f.create_table(f.root, "ratings", Ratings, filters=filters, expectedrows=len(ratings))
        table_ratings.append([ratings[col].values for col in ratings.ftypes.keys()])
        table_movies = f.create_table(f.root, "movies", Movies, filters=filters, expectedrows=len(movies))
        table_movies.append([movies[col].values for col in movies.ftypes.keys()])
    return filename

In [7]:
%%time
filters = tables.Filters(complevel=0, complib="zlib", shuffle=True)
h5file = to_hdf5(ratings, movies, filters)

Creating file: compression/no-compressed.h5
CPU times: user 27.5 ms, sys: 32.3 ms, total: 59.8 ms
Wall time: 80.8 ms


In [8]:
!h5ls -v {h5file}

Opened "compression/no-compressed.h5" with sec2 driver.
movies                   Dataset {3883/Inf}
    Attribute: CLASS scalar
        Type:      5-byte null-terminated UTF-8 string
        Data:  "TABLE"
    Attribute: FIELD_0_FILL scalar
        Type:      native int
        Data:  0
    Attribute: FIELD_0_NAME scalar
        Type:      8-byte null-terminated UTF-8 string
        Data:  "movie_id"
    Attribute: FIELD_1_FILL scalar
        Type:      1-byte null-terminated ASCII string
        Data:  ""
    Attribute: FIELD_1_NAME scalar
        Type:      5-byte null-terminated UTF-8 string
        Data:  "title"
    Attribute: FIELD_2_FILL scalar
        Type:      1-byte null-terminated ASCII string
        Data:  ""
    Attribute: FIELD_2_NAME scalar
        Type:      6-byte null-terminated UTF-8 string
        Data:  "genres"
    Attribute: NROWS scalar
        Type:      native long
        Data:  3883
    Attribute: TITLE null
        Type:      1-byte null-terminated UTF-8 

### Exercise 1

PyTables comes with out-of-box support for a series of codecs.  Do a quick comparison between "zlib", "bzip2", and "blosc" for compression levels of 1 (fastest), 5 and 9 (slowest).  Which one compresses best?  Which one compresses faster?

Also, Blosc being a meta-compressor, it has support for different codecs internally that can be selected from PyTables in the "blosc:`codec`" form.  Do another comparison between internal Blosc codecs, namely, "blosc:blosclz" (the default), "blosc:lz4", "blosc:lz4hc", "blosc:snappy", "blosc:zlib" and "blosc:zstd".

Finally, avoid any compression totally (`complevel=0`).  How fast it is compared with existing codecs?

In [9]:
for complib in ("zlib", "blosc:blosclz", "blosc:lz4", "blosc:lz4hc", "blosc:snappy", "blosc:zlib", "blosc:zstd"):
    filters = tables.Filters(complevel=5, complib=complib, shuffle=True)
    %time to_hdf5(ratings, movies, filters)

Creating file: compression/zlib-5-shuffle.h5
CPU times: user 378 ms, sys: 21.3 ms, total: 400 ms
Wall time: 426 ms
Creating file: compression/blosc-blosclz-5-shuffle.h5
CPU times: user 68.7 ms, sys: 12.4 ms, total: 81.1 ms
Wall time: 82.6 ms
Creating file: compression/blosc-lz4-5-shuffle.h5
CPU times: user 37 ms, sys: 11.9 ms, total: 48.9 ms
Wall time: 49.4 ms
Creating file: compression/blosc-lz4hc-5-shuffle.h5
CPU times: user 350 ms, sys: 15.5 ms, total: 366 ms
Wall time: 374 ms
Creating file: compression/blosc-snappy-5-shuffle.h5
CPU times: user 46.9 ms, sys: 13.3 ms, total: 60.3 ms
Wall time: 60.5 ms
Creating file: compression/blosc-zlib-5-shuffle.h5
CPU times: user 321 ms, sys: 15 ms, total: 336 ms
Wall time: 337 ms
Creating file: compression/blosc-zstd-5-shuffle.h5
CPU times: user 198 ms, sys: 14.6 ms, total: 213 ms
Wall time: 214 ms


## Reading compressed datasets

In [10]:
files = list(os.walk(data_dir))[0][2]

In [11]:
files

['blosc-blosclz-5-shuffle.h5',
 'blosc-lz4-5-shuffle.h5',
 'blosc-lz4hc-5-shuffle.h5',
 'blosc-snappy-5-shuffle.h5',
 'blosc-zlib-5-shuffle.h5',
 'blosc-zstd-5-shuffle.h5',
 'no-compressed.h5',
 'zlib-5-shuffle.h5']

In [12]:
for f in files:
    print("Reading file:", f)
    with tables.open_file(os.path.join(data_dir, f)) as h5f:
        %time h5f.root.ratings[:]

Reading file: blosc-blosclz-5-shuffle.h5
CPU times: user 42 ms, sys: 14.3 ms, total: 56.3 ms
Wall time: 76.2 ms
Reading file: blosc-lz4-5-shuffle.h5
CPU times: user 30.1 ms, sys: 10.6 ms, total: 40.7 ms
Wall time: 56.1 ms
Reading file: blosc-lz4hc-5-shuffle.h5
CPU times: user 23.9 ms, sys: 8.57 ms, total: 32.4 ms
Wall time: 33.2 ms
Reading file: blosc-snappy-5-shuffle.h5
CPU times: user 27.3 ms, sys: 7.84 ms, total: 35.1 ms
Wall time: 33.9 ms
Reading file: blosc-zlib-5-shuffle.h5
CPU times: user 86.2 ms, sys: 8.68 ms, total: 94.9 ms
Wall time: 97.8 ms
Reading file: blosc-zstd-5-shuffle.h5
CPU times: user 39.5 ms, sys: 6.2 ms, total: 45.7 ms
Wall time: 45.6 ms
Reading file: no-compressed.h5
CPU times: user 7.75 ms, sys: 10.6 ms, total: 18.4 ms
Wall time: 18.6 ms
Reading file: zlib-5-shuffle.h5
CPU times: user 77.2 ms, sys: 8.47 ms, total: 85.7 ms
Wall time: 90 ms


### Exercise 2

Which codec and compression level can read the fastest?  How it does compare with reading an uncompressed dataset?

### Exercise 3

Blosc can use multithreading for compressing/decompressing, although it is disabled by default.  You can enable a multithreaded Blosc in a series of ways, but perhaps the easiest is to set the "BLOSC_NTHREADS" environment variable to the desired number of threads (typically the available number of cores in your computer).

Execute the cell below and re-do the reading benchmarks and look at how the reading speed vary.  Pay special attention to the difference between the CPU times and wall times.

In [13]:
os.environ["BLOSC_NTHREADS"] = "4"  # set to any other number you prefer